In [2]:
import chromadb
import numpy as np
import pandas as pd

# Vector DB - Chroma DB 구축

In [2]:
df = pd.read_csv('날짜별_의정활동.csv')

In [6]:
client = chromadb.PersistentClient(path="../chromadb")

collection = client.get_or_create_collection(
    name="law_table",
    metadata={"hnsw:space": "cosine"}
)

In [4]:
# 인덱스
ids = []
# 메타 데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장한다고 한다. 
documents = []

In [5]:
for idx in range(len(df)):
    # iloc: index 위치를 기반으로 df에서 행을 선택
    item = df.iloc[idx]
    id = item['의안명'].replace(' ', '-')
    document = f"(의안명: {item['의안명']}) (의안 일자: {item['일자']}) (의안번호: {item['의안번호']}) - 단계:{item['단계']}, 세부단계: {item['세부단계']}, 소관위원회: {item['소관위원회']}"
    meta = {
        "카테고리": item['카테고리']
    }
    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장

In [6]:
collection.add(
    documents = documents,
    metadatas=doc_meta,
    ids=ids
)

# DB 쿼리

In [20]:
collection.query(
    query_texts=["주택"],
    n_results=5,
    where={"카테고리":"경제"},
    where_document={"$contains": "주택"}
)

{'ids': [['주택법-일부개정법률안(천준호의원등11인)',
   '주택법-일부개정법률안(천준호의원-등-12인)',
   '주택법-일부개정법률안(김정재의원등11인)',
   '주택법-일부개정법률안(이종배의원-등-11인)',
   '주택법-일부개정법률안(최인호의원-등-10인)']],
 'distances': [[0.6814720630645752,
   0.703223705291748,
   0.7037798166275024,
   0.7217906713485718,
   0.7317438125610352]],
 'metadatas': [[{'카테고리': '경제'},
   {'카테고리': '경제'},
   {'카테고리': '경제'},
   {'카테고리': '경제'},
   {'카테고리': '경제'}]],
 'embeddings': None,
 'documents': [['(의안명: 주택법 일부개정법률안(천준호의원등11인)) (의안 일자: 2023-11-22) (의안번호: 2120164) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 국토교통위원회',
   '(의안명: 주택법 일부개정법률안(천준호의원 등 12인)) (의안 일자: 2023-11-22) (의안번호: 2118465) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 국토교통위원회',
   '(의안명: 주택법 일부개정법률안(김정재의원등11인)) (의안 일자: 2023-11-22) (의안번호: 2116990) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 국토교통위원회',
   '(의안명: 주택법 일부개정법률안(이종배의원 등 11인)) (의안 일자: 2023-11-22) (의안번호: 2119207) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 국토교통위원회',
   '(의안명: 주택법 일부개정법률안(최인호의원 등 10인)) (의안 일자: 2023-11-22) (의안번호: 2121283) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 국토교통위원회']]}